In [2]:
import os 
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/shawnthomas0507/datascienceproj.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="shawnthomas0507"
os.environ['MLFLOW_TRACKING_PASSWORD']="2061b8589608029d512100109903e8a93d0b107d"

In [3]:
import os 
%pwd 

'c:\\Users\\shawn\\OneDrive\\Desktop\\Krish Naik MLOPS\\endtoend1\\datascienceproj\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\shawn\\OneDrive\\Desktop\\Krish Naik MLOPS\\endtoend1\\datascienceproj'

In [6]:
from dataclasses import dataclass
from pathlib import Path 


@dataclass
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path 
    model_path: Path 
    all_params: dict 
    metric_file_name: Path 
    target_column: str 
    mlflow_uri: str 

In [7]:
from src.datascience.constants import * 
from src.datascience.utils.common import read_yaml,create_directories,save_json

In [20]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])

    
    def get_model_eval_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri="https://dagshub.com/shawnthomas0507/datascienceproj.mlflow"
        )

        return model_evaluation_config


    


In [21]:
import pandas as pd 
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse 
import mlflow 
import mlflow.sklearn 
import numpy as np 
import joblib 

In [24]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config=config 

    def eval_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return rmse,mae,r2
    

    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store=urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            predicted_qualities=model.predict(test_x)

            (rmse,mae,r2_score)=self.eval_metrics(predicted_qualities,test_y)

            scores={"rmse":rmse,"mae":mae,"r2_score":r2_score}

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2_score",r2_score)


            if tracking_uri_type_store!="file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="Elastic model1")
            else:
                mlflow.sklearn.log_model(model,"model")
    

In [25]:
try:
        config=ConfigurationManager()
        model_eval_config=config.get_model_eval_config()
        eval=ModelEvaluation(model_eval_config)
        eval.log_into_mlflow()
    
except Exception as e:
        raise e

[2025-03-25 19:59:18,891: INFO common: yaml file: config\config.yaml loaded]
[2025-03-25 19:59:18,892: INFO common: yaml file: params.yaml loaded]
[2025-03-25 19:59:18,893: INFO common: yaml file: schema.yaml loaded]
[2025-03-25 19:59:18,895: INFO common: created directort at: artifacts]
[2025-03-25 19:59:18,895: INFO common: created directort at: artifacts/model_evaluation]


2025/03/25 19:59:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Elastic model1'.
2025/03/25 19:59:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Elastic model1, version 1


🏃 View run peaceful-snail-127 at: https://dagshub.com/shawnthomas0507/datascienceproj.mlflow/#/experiments/0/runs/92c017392b174832abb573caf51c8f54
🧪 View experiment at: https://dagshub.com/shawnthomas0507/datascienceproj.mlflow/#/experiments/0


Created version '1' of model 'Elastic model1'.
